###  EDA-3. Проектирование признаков 

In [4]:
import pandas as pd

# Прочитаем датасет
data = pd.read_csv('data/wine_cleared.csv')

In [5]:
# Создадим новый признак price_round, означающий округлённую до целого числа цену за бутылку вина:
# для удобства сразу преобразуем признак в int
data['price_round'] = data['price'].round().astype(int)

### Разбор текста

In [6]:
# Для того чтобы найти все числа в каждом значении серии, воспользуемся методом str.findall(). 
# Метод возвращает все совпадения с заданным шаблоном в серии pandas. 

regex = '\d{4}' # \d{4} означает четырехзначные числа в заданной строке.
data['year'] = data['title'].str.findall(regex).str.get(0)

In [7]:
# В наборе данных винных обзоров самая популярная страна-производитель вина — США. 
# Возможно, это не случайность, и факт производства в США влияет на рейтинг вина. Выделим этот факт. 
# Вы можете создать новый бинарный признак is_usa и присвоить ему 1 в случае, если вино произведено в США, иначе — 0.

data['is_usa'] = data['country'].apply(lambda x: 1 if x == 'US' else 0)

### Задание 2.1
Выберите из списка две самых популярных (помимо США) страны, производящих вино.

In [8]:
data['country'].value_counts()

US                        54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14
Serbia  

### Задание 2.2
Создайте бинарные признаки is_france, is_italy наподобие признака is_usa. 

In [9]:
data['is_france'] = data['country'].apply(lambda x: 1 if x == 'France' else 0)
data['is_italy'] = data['country'].apply(lambda x: 1 if x == 'Italy' else 0)

### Задание 2.3
Создайте новый бинарный признак old_wine, где значение 1 будет обозначать, что вино старше 2010 года.
В ответ впишите результат выполнения кода data['old_wine'].sum()

In [10]:
# преобразуем признак year в объект datetime для удобного сравнения дат
data['year'] = pd.to_datetime(data['year'], errors='coerce')
data['old_wine'] = data['year'].apply(lambda x: 1 if x.year < 2010 else 0) # x.year - чтобы вытянуть год из даты
display(data['old_wine'].sum())

39765

### Задание 2.7 (Самопроверка)
Создайте новый признак locality из признака title, который будет обозначать название долины/местности производства вина.
Например, в названии вина Rainstorm 2013 Pinot Gris (Willamette Valley) locality будет Willamette Valley. В названии Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra) — Navarra.

In [11]:
data['locality'] = data['title'].str.findall('\((\D*)\)').str.get(0)

# вариант 2:
#data['locality'] = data['title'].str.findall('\((.*?)\)').str.get(0)

### Работа с файлами

In [12]:
# прочтем файл
import pandas as pd
country_population = pd.read_csv('data/country_population.csv', sep=';')
country_population

,country,population
0,China,"1,411,778,724"
1,India,"1,386,584,581"
2,US,"333,022,386"
3,Indonesia,"271,350,000"
4,Pakistan,"225,200,000"
...,...,...
236,Niue,"1,549"
237,Tokelau,"1,501"
238,Vatican City,825
239,Cocos Islands,573


###  Задание 3.1
Каково население Италии согласно датасету country_population?

In [13]:
country_population[country_population['country'] == 'Italy']['population']

24    59,097,904
Name: population, dtype: object

In [14]:
# Объединим данные по населению стран с таблицей рейтингов вина
data = data.join(country_population.set_index('country'), on='country')

In [15]:
# Прочитаем файл с данными по площадам стран
import pandas as pd
country_area = pd.read_csv('data/country_area.csv', sep=';')
country_area

,country,area
0,Russia,17075400.00
1,Canada,9984670.00
2,China,9596960.00
3,US,9372610.00
4,Brazil,8547000.00
...,...,...
188,San Marino,61.00
189,Tuvalu,26.00
190,Nauru,21.00
191,Monaco,2.00


###  Задание 3.2
Создайте новый признак area_country — площадь страны, аналогичный признаку country_population.
Какая площадь страны у вина под названием 'Gård 2014 Grand Klasse Reserve Lawrence Vineyards Viognier (Columbia Valley (WA))'? Ответ вводите без пробелов, округлите до целых.

In [16]:
data = data.join(country_area.set_index('country'), on='country')

In [17]:
data[data['title'] == 'Gård 2014 Grand Klasse Reserve Lawrence Vineyards Viognier (Columbia Valley (WA))']['area']

94    9372610.0
Name: area, dtype: float64

###  Задание 4.5
Создайте признак количество дней с момента произведения вина — years_diff для датасета винных обзоров. За дату отсчёта возьмите 12 января 2022 года. В ответ впишите максимальное количество дней с момента произведения вина. Ответ округлите до целого числа.

In [22]:
data['years_diff'] = (pd.to_datetime('2022-01-12') - data['year']).dt.days

In [25]:
data['years_diff'].max()

98627.0